# Implementación de la metahuristica Busqueda Armonica Global GBHS

In [1]:
# Librerias y dependencias
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import distance
from sklearn.metrics import f1_score, accuracy_score
import sys
import math

In [2]:
sys.float_info.max

1.7976931348623157e+308

In [4]:
np.random.randint(5)

2

In [5]:
# Dataframes resultantes de la Fase 1
# Grupos de Calidad con su respctivo modelo y valor de correlación

GC1 = pd.read_excel("../Archivos Generados/PipelineResults/GruposCalidad-Fase2/Grupo1.xlsx",index_col=0)
GC2 = pd.read_excel("../Archivos Generados/PipelineResults/GruposCalidad-Fase2/Grupo2.xlsx",index_col=0)
GC3 = pd.read_excel("../Archivos Generados/PipelineResults/GruposCalidad-Fase2/Grupo3.xlsx",index_col=0)

In [6]:
print(f"Grupo de Calidad 1 {len(GC1)}, Grupo de Calidad 2: {len(GC2)}, Grupo de Calidad 3: {len(GC3)}")

Grupo de Calidad 1 398, Grupo de Calidad 2: 84, Grupo de Calidad 3: 317


In [7]:
# Se define la etiqueta de c/ grupo
GC1["Grupo"] = 1
GC2["Grupo"] = 2
GC3["Grupo"] = 3


In [8]:
# Se Unen los diferentes Grupos en un unico dataset
df = pd.concat([GC1, GC2,GC3], ignore_index=True)

In [9]:
df.shape

(799, 177)

In [10]:
# Se eliminan algunas variables no relevantes
df = df.drop(["ID_LOTE", "RDT_AJUSTADO"], axis=1)

In [11]:
df.head(5)

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA,Grupo
0,6,45,77,54000,11,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
1,5,54,80,60000,10,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
2,4,48,82,55000,12,1,0,0,0,0,...,1,0,0,0,1,0,0,1,0,1
3,5,47,73,70000,72,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1
4,5,47,81,67000,14,1,0,0,0,0,...,0,0,0,0,1,1,0,1,0,1


In [12]:
df.tail(5)

,DIAS_EN_EMERGER,DIAS_EN_EMERGER_A_FLORECER,DIAS_EN_FLORECER_A_COSECHAR,POBLACION_20DIAS_AJT,ALTURA_LOT,ContEnfQui_Emer_Flor,ContEnfQui_Flor_Cose,ContMalMec_Siem_Emer,ContMalMec_Emer_Flor,ContMalMec_Flor_Cose,...,MOTEADOS_RASTA,MOTEADOS_MAS70cm._RASTA,OBSERVA_EROSION_RASTA,OBSERVA_MOHO_RASTA,OBSERVA_RAICES_VIVAS_RASTA,OBSERVA_HOJARASCA_MO_RASTA,SUELO_NEGRO_BLANDO_RASTA,CUCHILLO_PRIMER_HTE_RASTA,CERCA_RIOS_QUEBRADAS_RASTA,Grupo
794,6,50,75,60000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
795,5,48,88,60000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
796,6,49,87,60000,17,0,0,0,0,0,...,0,0,0,0,1,1,0,1,0,3
797,7,47,84,70000,18,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3
798,6,49,79,61000,17,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,3


In [13]:
df.values[:]

array([[ 6., 45., 77., ...,  1.,  0.,  1.],
       [ 5., 54., 80., ...,  1.,  0.,  1.],
       [ 4., 48., 82., ...,  1.,  0.,  1.],
       ...,
       [ 6., 49., 87., ...,  1.,  0.,  3.],
       [ 7., 47., 84., ...,  1.,  0.,  3.],
       [ 6., 49., 79., ...,  1.,  0.,  3.]])

In [14]:
df.values[:,:-1]

array([[ 6., 45., 77., ...,  0.,  1.,  0.],
       [ 5., 54., 80., ...,  0.,  1.,  0.],
       [ 4., 48., 82., ...,  0.,  1.,  0.],
       ...,
       [ 6., 49., 87., ...,  0.,  1.,  0.],
       [ 7., 47., 84., ...,  0.,  1.,  0.],
       [ 6., 49., 79., ...,  0.,  1.,  0.]])

In [19]:
# Se realiza la normalización del dataset    [Todos los datos, exepto ultima columa de la clase del grupo]
norm = MinMaxScaler()
df_norm = norm.fit_transform(df.values[:,:-1])

In [20]:
df_norm

array([[0.4       , 0.3220339 , 0.46268657, ..., 0.        , 1.        ,
        0.        ],
       [0.33333333, 0.47457627, 0.50746269, ..., 0.        , 1.        ,
        0.        ],
       [0.26666667, 0.37288136, 0.53731343, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.4       , 0.38983051, 0.6119403 , ..., 0.        , 1.        ,
        0.        ],
       [0.46666667, 0.3559322 , 0.56716418, ..., 0.        , 1.        ,
        0.        ],
       [0.4       , 0.38983051, 0.49253731, ..., 0.        , 1.        ,
        0.        ]])

In [21]:
print(len(df_norm))

799


In [63]:
# El numero de ceros puede ser igual a la cuarta parte del total de caracteristicas
#nc = 50

def generarVectorPesos(w, nc):
    #w = np.random.rand(dim)
    posceros = np.random.choice(len(w), nc, replace=False)
    w[posceros] = 0
    #print(w)
    s  = np.sum(w)
    #print(s)
    # Se agregan 2 decimales
    wf = np.round(w/s, 3)
    #print(wf)
    sc = 1- np.sum(wf)
    #print(sc)
    pos = np.random.randint(len(wf))
    #print(pos)
    if sc != 0:
        wf[pos]= wf[pos]+sc

    return wf


In [64]:
pr = np.random.rand(10)
pr

array([0.54099617, 0.66954642, 0.35539109, 0.25686734, 0.8412412 ,
       0.01389162, 0.80075633, 0.88749626, 0.09220932, 0.01090736])

In [65]:
wff= generarVectorPesos(pr, 2)



In [66]:
wff

array([0.197, 0.244, 0.13 , 0.094, 0.   , 0.005, 0.292, 0.   , 0.034,
       0.004])

In [69]:
wp = np.random.rand(df_norm.shape[1])
generarVectorPesos(wp, 50)

array([0.016, 0.014, 0.   , 0.003, 0.013, 0.004, 0.001, 0.   , 0.002,
       0.   , 0.017, 0.005, 0.006, 0.012, 0.   , 0.013, 0.014, 0.001,
       0.002, 0.006, 0.   , 0.003, 0.   , 0.002, 0.011, 0.004, 0.   ,
       0.001, 0.006, 0.   , 0.   , 0.   , 0.014, 0.   , 0.007, 0.   ,
       0.004, 0.005, 0.   , 0.007, 0.015, 0.016, 0.   , 0.   , 0.01 ,
       0.017, 0.003, 0.005, 0.01 , 0.015, 0.015, 0.   , 0.016, 0.012,
       0.   , 0.004, 0.   , 0.   , 0.014, 0.009, 0.003, 0.004, 0.009,
       0.006, 0.001, 0.   , 0.   , 0.   , 0.009, 0.   , 0.   , 0.006,
       0.   , 0.   , 0.   , 0.004, 0.   , 0.012, 0.01 , 0.008, 0.006,
       0.   , 0.002, 0.   , 0.003, 0.004, 0.   , 0.007, 0.016, 0.   ,
       0.005, 0.003, 0.005, 0.   , 0.009, 0.   , 0.012, 0.   , 0.004,
       0.014, 0.004, 0.004, 0.   , 0.006, 0.005, 0.007, 0.012, 0.013,
       0.011, 0.003, 0.   , 0.005, 0.012, 0.003, 0.   , 0.008, 0.011,
       0.009, 0.012, 0.016, 0.   , 0.01 , 0.   , 0.008, 0.   , 0.006,
       0.012, 0.004,

### Construcción de la Función de Calidad (Qs)
**Parametros de entrada**
- df_norm: Dataset normalizado, sin tener en cunta la clase
- wi: Vector de pesos generado aleatroiamente

**Parametros de salida**
- Calidad (Qs) asociada a ese vector de pesos (wi)



In [70]:
# Dependeindo del vector de pesos me extrae el acuracy - F1 score
def qualityFunction(df_norm, wi):

    # Vector Distancias Euclideanas Ponderadas
    y_pred = []
    minDep = sys.float_info.max
    posMinDep = 0
    for i in range(len(df_norm)):
        vrf = df_norm[i] 
        for j in range(len(df_norm)):
            if i != j:
                ri = wi* np.power((df_norm[j] - vrf), 2)
                dE = np.sqrt(np.sum(ri))
                if dE < minDep:
                    posMinDep=j
                    minDep = dE
              
        y_pred.append(df.values[posMinDep][-1])
    qs = accuracy_score(df.values[:,-1], y_pred)
    return qs

### Construcción de la memoria Armonica
**Parametros de entrada**
- MAC: Numero de armonias que se van almacenar en la  memoria [5,10,15,20] 

**Parametros de salida**
-  MA : Memoria armonica ordenada deacuerdo al fitnes (Qs) de mayor a menor

In [74]:

def GenerateArmonyMemory(MAC):
    Lw = []
    for i in range (MAC):
        # 1. Generar pesos aleatorios | dim = cantidad de caracteristicas
        vp = np.random.rand(df_norm.shape[1])
        #2. Se genera el Vector de pesos normalizado
        wi = generarVectorPesos(vp, 50)
        #3. Se obtiene el fitnes asociados a esos pesos.
        Qs = qualityFunction(df_norm, wi)
        wiq = np.append(wi, Qs)
        Lw.append(wiq)

    
    # Se obtiene la memoria armonica ordenada de  Mayor- Menor
    #MA = Lw.sort(key=lambda x: x[-1], reverse=True)
    return Lw

In [91]:
# Genero la memoria Armonica
MA = GenerateArmonyMemory(5)


In [92]:
MA

[array([0.        , 0.011     , 0.        , 0.005     , 0.015     ,
        0.        , 0.        , 0.008     , 0.012     , 0.01      ,
        0.016     , 0.005     , 0.        , 0.011     , 0.012     ,
        0.005     , 0.        , 0.        , 0.        , 0.        ,
        0.01      , 0.001     , 0.006     , 0.002     , 0.005     ,
        0.        , 0.015     , 0.006     , 0.        , 0.012     ,
        0.003     , 0.        , 0.009     , 0.006     , 0.001     ,
        0.011     , 0.007     , 0.015     , 0.        , 0.        ,
        0.        , 0.002     , 0.        , 0.002     , 0.        ,
        0.        , 0.        , 0.014     , 0.        , 0.001     ,
        0.01      , 0.        , 0.        , 0.011     , 0.014     ,
        0.        , 0.016     , 0.011     , 0.003     , 0.005     ,
        0.005     , 0.        , 0.009     , 0.002     , 0.        ,
        0.002     , 0.011     , 0.        , 0.006     , 0.011     ,
        0.007     , 0.        , 0.013     , 0.00

In [93]:
for j in range(len(MA)):
    print(MA[j][0], end=' ')
    print(MA[j][1], end=' ')
    print(MA[j][2], end=' ')
    print("Fitnes: ",MA[j][-1])


0.0 0.011 0.0 Fitnes:  0.5769712140175219
0.001 0.0 0.0 Fitnes:  0.17897371714643304
0.0 0.011 0.0 Fitnes:  0.3979974968710889
0.004 0.006 0.0 Fitnes:  0.17897371714643304
0.011 0.0 0.011 Fitnes:  0.5757196495619524


In [95]:
# Ordeno la memoria Armonica
MA.sort(key=lambda x: x[-1], reverse=True)
MA

[array([0.        , 0.011     , 0.        , 0.005     , 0.015     ,
        0.        , 0.        , 0.008     , 0.012     , 0.01      ,
        0.016     , 0.005     , 0.        , 0.011     , 0.012     ,
        0.005     , 0.        , 0.        , 0.        , 0.        ,
        0.01      , 0.001     , 0.006     , 0.002     , 0.005     ,
        0.        , 0.015     , 0.006     , 0.        , 0.012     ,
        0.003     , 0.        , 0.009     , 0.006     , 0.001     ,
        0.011     , 0.007     , 0.015     , 0.        , 0.        ,
        0.        , 0.002     , 0.        , 0.002     , 0.        ,
        0.        , 0.        , 0.014     , 0.        , 0.001     ,
        0.01      , 0.        , 0.        , 0.011     , 0.014     ,
        0.        , 0.016     , 0.011     , 0.003     , 0.005     ,
        0.005     , 0.        , 0.009     , 0.002     , 0.        ,
        0.002     , 0.011     , 0.        , 0.006     , 0.011     ,
        0.007     , 0.        , 0.013     , 0.00

In [90]:
MA

[array([0.002     , 0.        , 0.013     , 0.003     , 0.011     ,
        0.        , 0.        , 0.006     , 0.006     , 0.001     ,
        0.012     , 0.004     , 0.        , 0.016     , 0.        ,
        0.011     , 0.004     , 0.01      , 0.        , 0.01      ,
        0.002     , 0.008     , 0.014     , 0.        , 0.        ,
        0.016     , 0.003     , 0.005     , 0.006     , 0.002     ,
        0.016     , 0.009     , 0.005     , 0.015     , 0.004     ,
        0.01      , 0.006     , 0.004     , 0.016     , 0.016     ,
        0.        , 0.        , 0.002     , 0.004     , 0.014     ,
        0.008     , 0.        , 0.009     , 0.012     , 0.001     ,
        0.        , 0.017     , 0.002     , 0.008     , 0.006     ,
        0.        , 0.007     , 0.008     , 0.        , 0.011     ,
        0.        , 0.008     , 0.002     , 0.011     , 0.003     ,
        0.012     , 0.        , 0.007     , 0.009     , 0.009     ,
        0.013     , 0.        , 0.006     , 0.  

### Implementacion GBHS
**Parametros de entrada**
- Imp: Numero de improvisaciones (iteraciones) que realiza GBHS.
- ParMin: Tasa de ajuste de tono mínima.
- ParMax: Tasa de ajuste de tono máxima.
- HCMR: Tasa de consideración de memoria armónica.
- p: Numero de atributos dataset Original o Normalizado



In [94]:
lmp = 10
P= len(MA[0])
HMRC = 0.85
PAR = 0.35              #[0.1, 0.12, 0.13 , 0.40]
# Tamaño de la memoria armonica
hmns = 5

In [98]:
len(MA[0])

175

In [100]:
import random

curva = []
for i in range (lmp):
    print(f"Iteracion {i}")
    pesosAleatorios = np.random.rand(P-1)
    # print("Vector de Pesos: ", pesosAleatorios)
    # Donde P es el numero de variables
    for j in range(P-1):
        Aleatorio1 = random.random() 
        if (Aleatorio1 < HMRC):
            # pma: Numero entrero entre 0 y (hmns -1)
            # hmns: Tamaño de la memoria armonica
            pma = random.randint(0, hmns-1)
            pesosAleatorios[j]= MA[pma][j]

            Aleatorio2 = random.random()
            if Aleatorio2 < PAR:
                # Se toma el valor de la mejor armonia
                pesosAleatorios[j] = MA[0][j]
        
        else:

            Aleatorio3 = random.random()/(P/2)
            pesosAleatorios[j] = Aleatorio3
    
    # Fin primer for - Conformación vector de pesosS
    wf = generarVectorPesos(pesosAleatorios,50)
    fitnes = qualityFunction(df_norm, wf)
    print("Funcion de calidad: ", fitnes)


    # Remplazo

    if MA[hmns -1][P-1] < fitnes:
        # Creo el nuevo registro y  remplazo por el peor
        new_register = np.append(wf, fitnes)
        # Remplazo
        MA[hmns-1] = new_register
        # Ordeno la Armonia
        print("------------------------------------")
        MA.sort(key=lambda x: x[-1], reverse=True)
    
    else:
        print("La funcion de calidad del ultimo armonico  es mejor que el fitnes")
    
    curva.append(MA[0][P-1])
    


    


Iteracion 0
Funcion de calidad:  0.5782227784730913
------------------------------------
Iteracion 1
Funcion de calidad:  0.5682102628285357
------------------------------------
Iteracion 2
Funcion de calidad:  0.8397997496871089
------------------------------------
Iteracion 3
Funcion de calidad:  0.5807259073842302
------------------------------------
Iteracion 4
Funcion de calidad:  0.7146433041301627
------------------------------------
Iteracion 5
Funcion de calidad:  0.29912390488110135
La funcion de calidad del ultimo armonico  es mejor que el fitnes
Iteracion 6
Funcion de calidad:  0.8297872340425532
------------------------------------
Iteracion 7


C:\Users\germanm\AppData\Local\Temp\ipykernel_7232\219039592.py:13: RuntimeWarning: invalid value encountered in sqrt
  dE = np.sqrt(np.sum(ri))


Funcion de calidad:  0.8222778473091364
------------------------------------
Iteracion 8
Funcion de calidad:  0.8435544430538173
------------------------------------
Iteracion 9


C:\Users\germanm\AppData\Local\Temp\ipykernel_7232\219039592.py:13: RuntimeWarning: invalid value encountered in sqrt
  dE = np.sqrt(np.sum(ri))


Funcion de calidad:  0.8498122653316645
------------------------------------


In [101]:
curva

[0.5782227784730913,
 0.5782227784730913,
 0.8397997496871089,
 0.8397997496871089,
 0.8397997496871089,
 0.8397997496871089,
 0.8397997496871089,
 0.8397997496871089,
 0.8435544430538173,
 0.8498122653316645]

In [39]:
MA

[array([ 0.001     ,  0.007     ,  0.001     ,  0.009     ,  0.01      ,
         0.005     ,  0.006     ,  0.002     ,  0.001     ,  0.002     ,
         0.002     ,  0.01      ,  0.006     ,  0.009     ,  0.002     ,
         0.01      ,  0.009     ,  0.01      ,  0.009     ,  0.007     ,
         0.        ,  0.002     ,  0.009     ,  0.006     ,  0.004     ,
         0.035     ,  0.        ,  0.        ,  0.009     ,  0.006     ,
         0.008     ,  0.009     ,  0.002     ,  0.004     ,  0.001     ,
         0.01      ,  0.007     ,  0.005     ,  0.        ,  0.004     ,
         0.005     ,  0.003     ,  0.009     ,  0.003     ,  0.        ,
         0.012     ,  0.007     ,  0.007     ,  0.007     ,  0.007     ,
         0.007     ,  0.004     ,  0.        ,  0.        ,  0.006     ,
         0.009     ,  0.009     ,  0.004     ,  0.003     ,  0.002     ,
         0.005     ,  0.002     ,  0.004     ,  0.007     ,  0.01      ,
         0.        ,  0.009     ,  0.001     ,  0.0

In [ ]:
curva

In [75]:
c = np.array([1.3,4])

In [77]:
d = np.append(c, 5)

In [78]:
d

array([1.3, 4. , 5. ])

In [71]:
MA[0][174]

0.5031289111389237